In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wave
from scipy import signal
from scipy.io import wavfile
import IPython.display as ipd
import librosa
import librosa.display

from python_speech_features import fbank
from audio2numpy import open_audio
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
# load raw data
data=pd.read_csv("/Users/anqiliu/Desktop/SACC/archive/speakers_all.csv",index_col='speakerid')
data = data.iloc[:,:8]
data = data[data['file_missing?']==False]
data

In [ ]:
INPUT_DIR = '/Users/anqiliu/Desktop/SACC/archive/recordings/recordings'
OUTPUT_DIR = 'results'
parent_list = os.listdir(INPUT_DIR)
print(len(parent_list))

print(set(data['filename']+'.mp3') - set(parent_list)) # find contries whose recordings are misssing

data = data.drop(data[data['filename']=='sinhalese1'].index)
data = data.drop(data[data['filename']=='nicaragua'].index)
data

In [ ]:
# 4 classification
English = data[data['native_language']=='english']
French = data[data['native_language']=='french']
Spanish= data[data['native_language']=='spanish']
Arabic = data[data['native_language']=='arabic']
dataSub = English.append(French).append(Spanish).append(Arabic)
dataSub

In [ ]:
def get_mfcc(INPUT_DIR):  
    parent_list = dataSub['filename']+ '.mp3'
    # 2138 recordings each has 20 862-dimension MFCCs (dim depends on duration)
    mfcc_feat = np.zeros((len(parent_list),20,862))
    i=0  
    for file in parent_list[:]:
        f_name = str(INPUT_DIR+file)
        y, sr = librosa.load(f_name,duration = 20)    # duration 20s, SAMPLE_RATE = 22050 default
        mfcc = librosa.feature.mfcc(y=y, sr=sr) 
 
        if len(mfcc[1]) < 862:
            offset = 862 - len(mfcc[1]) # padding starting point
            mfcc= np.pad(mfcc,((0,0),(offset,0)), 'constant')

        mfcc_feat[i,:,:]=mfcc
        i+=1
        
    return mfcc_feat

In [ ]:
def get_fbank(INPUT_DIR):
    parent_list = dataSub['filename']+ '.mp3'
    # 2138 recordings each has 20 3990-dimension Fbanks (dim depends on duration)
    fbank_feat = np.zeros((len(parent_list),20,3990))
    i=0  
    for file in parent_list[:]:
        f_name = str(INPUT_DIR+file)
        y, sr = librosa.load(f_name,sr=None,duration=20)   # duration 20s, uses the native sampling rate
        fb = fbank(y,sr/2,nfft=1103,nfilt=20)[0].T
        
        if len(fb[1]) < 3990:
            offset = 3990 - len(fb[1]) # padding starting point
            fb= np.pad(fb,((0,0),(offset,0)), 'constant')
   
        if len(fb[1]) > 3990:
            fb = fb[:,:3990]    # extract the first 3990 dims
        
        fbank_feat[i,:,:]= fb
        i+=1
        
    return fbank_feat

In [ ]:
INPUT_DIR = '/Users/anqiliu/Desktop/SACC/archive/recordings/recordings'
mfcc_feat = get_mfcc(INPUT_DIR)
np.save("/Users/anqiliu/Desktop/SACC/Results/mfcc_feat4.npy",mfcc_feat)

In [ ]:
fbank_feat = get_fbank(INPUT_DIR)
np.save("/Users/anqiliu/Desktop/SACC/Results/fbank_feat4.npy",fbank_feat)